In [1]:
import os
import pandas as pd
import requests
import re
import openpyxl
import time
from selenium import webdriver
from bs4 import BeautifulSoup 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.common.by import By

In [2]:
os.getcwd()

'C:\\Users\\samsung\\Desktop\\강의 등\\barogo\\요기요크롤링'

In [4]:
options = Options()

# chrome을 전체화면으로 넓히는 옵션 => headless에서 적용 안되는 문제
#options.add_argument('--start-fullscreen') 
# 크롬창을 띄우지 않는 옵션을 넣는다	
#options.add_argument('headless')	
options.add_argument('window-size=1920,1080')

In [51]:
# 한 개 테스트 코드

# time sleep용
PAUSE_TIME = 2

# 저장 파일
f = open('요기요크롤링.csv', 'w')
f.write('법정동명,카테고리,상호명,평점,리뷰수,사장님댓글수,최소주문금액,광고여부\n')

# 광고 구분용 dict
d = dict()
d[2] = '우리동네플러스'
d[3] = '슈퍼레드위크추천'
d[4] = '요기요등록음식점'

# 드라이버 변수를 만듭니다. / 크롬을 켭니다.
driver = webdriver.Chrome('./chromedriver', options=options)  

driver.get('https://www.yogiyo.co.kr/')

key = '신수동'
time.sleep(PAUSE_TIME)

elem = driver.find_element_by_xpath('//*[@id="search"]/div/form/input')
elem.clear()
elem.send_keys('서울시 ' + key + ' 주민센터')  # 해당 요소에 문자 입력
time.sleep(PAUSE_TIME)
elem.send_keys(Keys.RETURN) # 엔터키
time.sleep(PAUSE_TIME)

for num in range(3, 15):
    time.sleep(PAUSE_TIME)
    # 카테고리 클릭
    body = driver.find_element_by_xpath('//*[@id="category"]/ul/li[{}]'.format(num))
    body.click()
    
    # 카테고리명
    category = driver.find_element_by_xpath('//*[@id="category"]/ul/li[{}]/span'.format(num))
    #print(category.text)
    
    time.sleep(PAUSE_TIME)
    
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # Scroll down to bottom                                                      
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);") 
        time.sleep(PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height         
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:         
            break
        
        last_height = new_height
    
    
    # 가게명/평점/리뷰/댓글 추출
    for box_num in range(2, 5):
        sub_num = 1
        time.sleep(PAUSE_TIME)
        
        while True:
            time.sleep(PAUSE_TIME)
            try:
                cont_add = '//*[@id="content"]/div/div[{}]/div/div[2]/div[{}]/div/table/tbody/tr/td[2]'\
                            .format(box_num, sub_num) 
                
                '''
                # 상호명
                print(driver.find_element_by_xpath(cont_add + '/div/div[1]').text)

                # 평점
                print(driver.find_element_by_xpath(cont_add + '/div/div[2]/span[1]/span').text.replace('★ ', ''))
                
                # 리뷰수
                print(driver.find_element_by_xpath(cont_add + '/div/div[2]/span[2]').text.replace('리뷰 ', ''))
                
                # 사장님댓글수
                print(driver.find_element_by_xpath(cont_add + '/div/div[2]/span[3]').text.replace('사장님댓글 ', ''))
                
                # 최소주문금액
                print(driver.find_element_by_xpath(cont_add + '/div/ul/li[2]').text.replace('원 이상 배달', '').replace(',', ''))
                '''
                
                # 파일에 쓰기
                f.write(key + ',' +
                        category.text + ',' + 
                        driver.find_element_by_xpath(cont_add + '/div/div[1]').text + ',' +
                        driver.find_element_by_xpath(cont_add + '/div/div[2]/span[1]/span').text.replace('★ ', '') + ',' +
                        driver.find_element_by_xpath(cont_add + '/div/div[2]/span[2]').text.replace('리뷰 ', '') + ',' +
                        driver.find_element_by_xpath(cont_add + '/div/div[2]/span[3]').text.replace('사장님댓글 ', '') + ',' +
                        driver.find_element_by_xpath(cont_add + '/div/ul/li[2]').text.replace('원 이상 배달', '').replace(',', '') + ',' +
                        d[box_num] + '\n') 
                
                sub_num += 1
            except:
                break      
                
f.close()

KeyboardInterrupt: 

In [39]:
'''
text = pd.read_csv('latlng_bdong.txt', sep='\t')
text[['위도', '경도']] = text.좌표.apply(lambda x: pd.Series(str(x).split(','))) 
text.to_csv('법정동_중심좌표.csv', encoding='cp949', index=False)
'''

In [5]:
# 법정동 중심좌표 불러오기

address = pd.read_csv('법정동_중심좌표_수정.csv', encoding='cp949')
address.head()

,No,코드,구분,시도,시군구,읍면동,하위,좌표,위도,경도,지번주소,도로명주소
0,1,1111010100,296,서울특별시,종로구,청운동,NaN,"37.587111,126.969069",37.587111,126.969069,서울특별시 종로구 청운동 53-55,서울특별시 종로구 자하문로 125-7
1,2,1111010200,178,서울특별시,종로구,신교동,NaN,"37.583911,126.968354",37.583911,126.968354,서울특별시 종로구 신교동 6-1,서울특별시 종로구 필운대로 100
2,3,1111010300,20,서울특별시,종로구,궁정동,NaN,"37.584381,126.971489",37.584381,126.971489,서울특별시 종로구 궁정동 17-3,서울특별시 종로구 자하문로26길 10
3,4,1111010400,131,서울특별시,종로구,효자동,NaN,"37.582416,126.971670",37.582416,126.971670,서울특별시 종로구 효자동 50-6,서울특별시 종로구 자하문로24길 38
4,5,1111010500,126,서울특별시,종로구,창성동,NaN,"37.580363,126.972065",37.580363,126.972065,서울특별시 종로구 창성동 108-11,서울특별시 종로구 자하문로12길 23


In [6]:
address.shape

(18673, 12)

In [6]:
address_seoul = address[address.시도 == '서울특별시']
address_seoul.head()

,No,코드,구분,시도,시군구,읍면동,하위,좌표,위도,경도,지번주소,도로명주소
0,1,1111010100,296,서울특별시,종로구,청운동,NaN,"37.587111,126.969069",37.587111,126.969069,서울특별시 종로구 청운동 53-55,서울특별시 종로구 자하문로 125-7
1,2,1111010200,178,서울특별시,종로구,신교동,NaN,"37.583911,126.968354",37.583911,126.968354,서울특별시 종로구 신교동 6-1,서울특별시 종로구 필운대로 100
2,3,1111010300,20,서울특별시,종로구,궁정동,NaN,"37.584381,126.971489",37.584381,126.971489,서울특별시 종로구 궁정동 17-3,서울특별시 종로구 자하문로26길 10
3,4,1111010400,131,서울특별시,종로구,효자동,NaN,"37.582416,126.971670",37.582416,126.971670,서울특별시 종로구 효자동 50-6,서울특별시 종로구 자하문로24길 38
4,5,1111010500,126,서울특별시,종로구,창성동,NaN,"37.580363,126.972065",37.580363,126.972065,서울특별시 종로구 창성동 108-11,서울특별시 종로구 자하문로12길 23


In [8]:
address_seoul.shape

(467, 12)

In [7]:
# 서울특별시 법정동명-도로명주소 dictionary 생성
# => key값으로 법정동명 들어갔을 때, 중심 도로명주소 검색하도록 만들기

add_dict = dict()

for i in range(address_seoul.shape[0]):
    add_dict[address_seoul.iloc[i, 5]] = address_seoul.iloc[i, 11]
    
add_dict

{'청운동': '서울특별시 종로구 자하문로 125-7',
 '신교동': '서울특별시 종로구 필운대로 100',
 '궁정동': '서울특별시 종로구 자하문로26길 10',
 '효자동': '서울특별시 종로구 자하문로24길 38',
 '창성동': '서울특별시 종로구 자하문로12길 23',
 '통의동': '서울특별시 종로구 자하문로6길 11-8',
 '적선동': '서울특별시 종로구 사직로 133-6',
 '통인동': '서울특별시 종로구 자하문로13길 13-3',
 '누상동': '서울특별시 종로구 옥인3길 21',
 '누하동': '서울특별시 종로구 필운대로5길 4',
 '옥인동': '서울특별시 종로구 필운대로9길 5',
 '체부동': '서울특별시 종로구 자하문로5길 15',
 '필운동': '서울특별시 종로구 필운대로1길 7-7',
 '내자동': '서울특별시 종로구 사직로12길 1-1',
 '사직동': '서울특별시 종로구 사직로 62',
 '도렴동': '서울특별시 종로구 세종로 세종대로 175',
 '당주동': '서울특별시 종로구 새문안로5길 13',
 '내수동': '서울특별시 종로구 경희궁2길 14',
 '세종로': '서울특별시 종로구 세종로 세종대로 175',
 '신문로1가': '서울특별시 종로구 신문로1가 새문안로 83',
 '신문로2가': '서울특별시 종로구 신문로2가 새문안로 45',
 '청진동': '서울특별시 종로구 종로5길 32-3',
 '서린동': '서울특별시 종로구 종로 24-4',
 '수송동': '서울특별시 종로구 종로5길 76',
 '중학동': '서울특별시 종로구 종로1길 58',
 '종로1가': '서울특별시 종로구 종로5길 7',
 '공평동': '서울특별시 종로구 우정국로 29',
 '관훈동': '서울특별시 종로구 인사동길 44',
 '견지동': '서울특별시 종로구 우정국로 52',
 '와룡동': '서울특별시 종로구 율곡로10길 13',
 '권농동': '서울특별시 종로구 서순라길 121',
 '운니동': '서울특별시 종로구 율곡로6길 26',
 '익선

In [12]:
list(add_dict.keys())[353]

'내발산동'

In [13]:
# 전체 크롤링 코드

# time sleep용
PAUSE_TIME = 2

# 저장 파일
f = open('요기요크롤링_서울.csv', 'a')
#f.write('법정동명,카테고리,상호명,평점,리뷰수,사장님댓글수,최소주문금액,광고여부\n')

# 광고 구분용 dict
d = dict()
d[2] = '우리동네플러스'
d[3] = '슈퍼레드위크추천'
d[4] = '요기요등록음식점'

# 드라이버 변수를 만듭니다. / 크롬을 켭니다.
driver = webdriver.Chrome('./chromedriver', options=options)  

driver.get('https://www.yogiyo.co.kr/')

count = 354

for key in list(add_dict.keys())[353:]:
    print('\n------{}({}):{}------'.format(key, count, add_dict[key]))
    time.sleep(PAUSE_TIME)
    
    elem = driver.find_element_by_xpath('//*[@id="search"]/div/form/input')
    elem.clear()
    elem.send_keys(add_dict[key])  # 해당 요소에 문자 입력
    
    # 검색 바로 안되는 경우
    try:
        elem.send_keys(Keys.RETURN) # 엔터키
        time.sleep(PAUSE_TIME)
        another_case = driver.find_element_by_xpath('//*[@id="search"]/div/form/ul/li[3]/a')
        another_case.click()
        time.sleep(PAUSE_TIME)
        elem.send_keys(Keys.RETURN) # 엔터키
        time.sleep(PAUSE_TIME)
    # 검색 바로 되는 경우
    except:
        time.sleep(PAUSE_TIME)
        elem.send_keys(Keys.RETURN) # 엔터키
        time.sleep(PAUSE_TIME)
        
    for num in range(3, 15):
        time.sleep(PAUSE_TIME)
        # 카테고리 클릭
        try:
            body = driver.find_element_by_xpath('//*[@id="category"]/ul/li[{}]'.format(num))
            body.click()
        except:
            print('click error!')
        
        # 카테고리명
        category = driver.find_element_by_xpath('//*[@id="category"]/ul/li[{}]/span'.format(num))
        print(category.text)

        time.sleep(PAUSE_TIME)

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom                                                      
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            try:
                time.sleep(PAUSE_TIME)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);") 
                time.sleep(PAUSE_TIME)
            except:
                print('scroll down error!')
                
            # Calculate new scroll height and compare with last scroll height         
            new_height = driver.execute_script("return document.body.scrollHeight")

            if new_height == last_height:         
                break

            last_height = new_height


        # 가게명/평점/리뷰/댓글 추출
        for box_num in range(2, 5):
            sub_num = 1
            time.sleep(PAUSE_TIME)

            while True:
                time.sleep(PAUSE_TIME)
                try:
                    cont_add = '//*[@id="content"]/div/div[{}]/div/div[2]/div[{}]/div/table/tbody/tr/td[2]'\
                                .format(box_num, sub_num) 

                    # 파일에 쓰기
                    f.write(key + ',' +
                            category.text + ',' + 
                            driver.find_element_by_xpath(cont_add + '/div/div[1]').text + ',' +
                            driver.find_element_by_xpath(cont_add + '/div/div[2]/span[1]/span').text.replace('★ ', '') + ',' +
                            driver.find_element_by_xpath(cont_add + '/div/div[2]/span[2]').text.replace('리뷰 ', '') + ',' +
                            driver.find_element_by_xpath(cont_add + '/div/div[2]/span[3]').text.replace('사장님댓글 ', '') + ',' +
                            driver.find_element_by_xpath(cont_add + '/div/ul/li[2]').text.replace('원 이상 배달', '').replace(',', '') + ',' +
                            d[box_num] + '\n') 

                    sub_num += 1
                except:
                    break      
    count += 1

f.close()


------내발산동(354):서울특별시 강서구 강서로47가길 3------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------외발산동(355):서울특별시 강서구 남부순환로19길 9------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------공항동(356):서울특별시 강서구 남부순환로11가길 117------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------방화동(357):서울특별시 강서구 금낭화로6가길 7------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------개화동(358):서울특별시 강서구 개화동로5길 19------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------과해동(359):서울특별시 강서구 하늘길 74------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------오곡동(360):서울특별시 강서구 개화동로2길 974------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------오쇠동(361):서울특별시 강서구 대장로 39------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------신도림동(362):서울특별시 구로구 신도림로11길 8-26------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------구로동(363):서울특별시 구로구 가마산로20나길 3------
1인분 주문
프랜차이즈
치킨

1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------역삼동(431):서울특별시 강남구 테헤란로 202------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------개포동(432):서울특별시 강남구 개포로38길 24------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------청담동(433):서울특별시 강남구 삼성로149길 8------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------삼성동(434):서울특별시 강남구 삼성로 575------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------대치동(435):서울특별시 강남구 삼성로69길 16------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------논현동(436):서울특별시 강남구 논현로 662------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------압구정동(437):서울특별시 강남구 압구정로 222------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------세곡동(438):서울특별시 강남구 헌릉로571길 20------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------자곡동(439):서울특별시 강남구 자곡로 175------
1인분 주문
프랜차이즈
치킨
피자/양식
중국집
한식
일식/돈까스
족발/보쌈
야식
분식
카페/디저트
편의점/마트

------율